In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models
import keras
import numpy as np
import cv2
import os
import pandas as pd


In [2]:
# -*- coding: UTF-8 -*-
# https://blog.csdn.net/u013832707/article/details/73608504
import sys

class ShowProcess():
    """
    显示处理进度的类
    调用该类相关函数即可实现处理进度的显示
    """
    i = 0 # 当前的处理进度
    max_steps = 0 # 总共需要处理的次数
    max_arrow = 50 #进度条的长度
    infoDone = 'done'

    # 初始化函数，需要知道总共的处理次数
    def __init__(self, max_steps, infoDone = 'Done'):
        self.max_steps = max_steps
        self.i = 0
        self.infoDone = infoDone

    # 显示函数，根据当前的处理进度i显示进度
    # 效果为[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>]100.00%
    def show_process(self, i=None):
        if i is not None:
            self.i = i
        else:
            self.i += 1
        num_arrow = int(self.i * self.max_arrow / self.max_steps) #计算显示多少个'>'
        num_line = self.max_arrow - num_arrow #计算显示多少个'-'
        percent = self.i * 100.0 / self.max_steps #计算完成进度，格式为xx.xx%
        process_bar = '[' + '>' * num_arrow + '-' * num_line + ']'\
                      + '%.2f' % percent + '%' + '\r' #带输出的字符串，'\r'表示不换行回到最左边
        sys.stdout.write(process_bar) #这两句打印字符到终端
        sys.stdout.flush()
        if self.i >= self.max_steps:
            self.close()

    def close(self):
        print('')
        print(self.infoDone)
        self.i = 0


### 1 读取模型

In [3]:
# 读取模型model
model_whale = keras.models.load_model('whale-model-2023-6-14.h5')
model_ship = keras.models.load_model('ship-model-2023-6-14.h5')
# 打印模型概述
# model_whale.summary()
# model_ship.summary()

In [23]:
# image_data=pd.read_csv('D:/workshop/2023-6-12/2-test/ship/train/train.csv').image.values

In [25]:

# source_url='D:/workshop/2023-6-12/2-test/ship-resize32/'
# process_bar = ShowProcess(6252, 'Processing Finished')
# for i in range(6252):
#     i_path=source_url+image_data[i]
#     image_data[i]=cv2.resize(cv2.imread(i_path,cv2.IMREAD_GRAYSCALE),(32,32))
#     process_bar.show_process()
# train_images_staked = np.stack(image_data, axis=0)
# print(train_images_staked.shape)

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>]100.00%
Processing Finished
(6252, 32, 32)


In [32]:
# object='ship'
# # 将Keras模型转换为TfLite模型
# converter_quant = tf.lite.TFLiteConverter.from_keras_model(model_ship)
# # set the optimization parameters for TensorFlow Lite conversion
# converter_quant.optimizations = [tf.lite.Optimize.DEFAULT]
# converter_quant.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8] 
# # images=tf.convert_to_tensor(train_images_staked)
# images = tf.cast(train_images_staked,tf.float32) / 255.0
# mnist_ds = tf.data.Dataset.from_tensor_slices((images)).batch(1)
# def representative_data_gen():
#   for input_value in mnist_ds.take(100):
#     yield [input_value]
# converter_quant.representative_dataset = representative_data_gen
# # convert the model to TensorFlow Lite format with float32 activations and int8 weights
# quanitfied_model = converter_quant.convert()
# #保存转换后的模型
# quanitfied_name = object+"-model-2023-6-14-quantization.tflite"
# open('./tflite_model/'+quanitfied_name, "wb").write(quanitfied_model)
# print(object,'quantization size',np.round(os.path.getsize('./tflite_model/'+quanitfied_name)/1000,3),'KB')

INFO:tensorflow:Assets written to: D:\USER_T~1\tmpl8mfx6kz\assets


INFO:tensorflow:Assets written to: D:\USER_T~1\tmpl8mfx6kz\assets


ship quantization size 274.768 KB


d:\Anaconda\envs\tensorflow\lib\site-packages\tensorflow\lite\python\convert.py:765: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn("Statistics for quantized inputs were expected, but not "


### 2 模型转换与量化

In [16]:
def model_transform(model,object):
    #训练一个keras模型并转换为tflite格式
    #源float模型
    converter = tf.lite.TFLiteConverter.from_keras_model(model)
    tflite_model = converter.convert()
    tflite_name = object+"-model-2023-6-14.tflite"
    open('./tflite_model/'+tflite_name, "wb").write(tflite_model)
    print(object,'tflite size',np.round(os.path.getsize('./tflite_model/'+tflite_name)/1000,3),'KB')
    
def model_quantization(model,object):
    # 将Keras模型转换为TfLite模型
    converter_quant = tf.lite.TFLiteConverter.from_keras_model(model)
    # set the optimization parameters for TensorFlow Lite conversion
    converter_quant.optimizations = [tf.lite.Optimize.DEFAULT]

    # convert the model to TensorFlow Lite format with float32 activations and int8 weights
    quanitfied_model = converter_quant.convert()

    #保存转换后的模型
    quanitfied_name = object+"-model-2023-6-14-quantization.tflite"
    open('./tflite_model/'+quanitfied_name, "wb").write(quanitfied_model)
    print(object,'quantization size',np.round(os.path.getsize('./tflite_model/'+quanitfied_name)/1000,3),'KB')

In [28]:
model_transform(model_whale,'whale') # whale 转换

INFO:tensorflow:Assets written to: D:\USER_T~1\tmpszrz6edd\assets


INFO:tensorflow:Assets written to: D:\USER_T~1\tmpszrz6edd\assets


whale tflite size 4233.356 KB


In [29]:
model_quantization(model_whale,'whale') # whale 量化

INFO:tensorflow:Assets written to: D:\USER_T~1\tmp0n0y29zb\assets


INFO:tensorflow:Assets written to: D:\USER_T~1\tmp0n0y29zb\assets


whale quantization size 1063.208 KB


In [30]:
model_transform(model_ship,'ship') # ship 转换

INFO:tensorflow:Assets written to: D:\USER_T~1\tmpg_j4oiaf\assets


INFO:tensorflow:Assets written to: D:\USER_T~1\tmpg_j4oiaf\assets


ship tflite size 1075.324 KB


In [31]:
model_quantization(model_ship,'ship') # ship 量化

INFO:tensorflow:Assets written to: D:\USER_T~1\tmp8b9r_xf1\assets


INFO:tensorflow:Assets written to: D:\USER_T~1\tmp8b9r_xf1\assets


ship quantization size 275.56 KB


### 3 模型测试

##### 3.1 原模型

3.1.1 whale

In [17]:
# 读取模型model
# model_whale = keras.models.load_model('D:/workshop/2023-6-12/5-Version/1-whale/whale-model-2023-6-10.h5')
model_whale=keras.models.load_model('whale-model-2023-6-15.h5')

In [18]:
image_label_30=list() # 创建空列表，用于存储图像-标签对

f=open('test_image_label_6_13.txt','r')
for i in f.readlines():
    image_label_30.append(i.split('\n')[0]) # 读取图像-标签对
f.close()
for i in range(len(image_label_30)):
    image_label_30[i]=image_label_30[i].split(' ') # 预处理图像-标签对

In [19]:
def model_test(image_name,label):
    # 加载测试数据并进行预处理
    i_path='./whale64x64/'+image_name
    test_data = cv2.resize(cv2.imread(i_path,cv2.IMREAD_GRAYSCALE),(32,32))
    # test_data = cv2.imread(i_path,cv2.IMREAD_GRAYSCALE).astype('float32')
    test_data=np.expand_dims(test_data, axis=-1)
    # 调整输入数据的形状
    if len(test_data.shape) == 3:
        # 单个图像，形状为(64, 64, 1)
        test_data = np.expand_dims(test_data, axis=0)
    elif len(test_data.shape) == 4:
        # 多个图像，形状为(N, 64, 64, 1)
        pass  # 数据已经符合要求
    else:
        raise ValueError("Invalid input shape")
    # 进行预测
    predictions = model_whale.predict(test_data, verbose=0)
    # 判断结果正误
    if(predictions.argmax()==int(label)):
        print(image_name,label)
        return(1)
    else:
        return(0)

In [ ]:
# 测试集
ADD_accurate=0
process_bar = ShowProcess(len(image_label_30), 'Processing Finished')
for i in range(len(image_label_30)):
   ADD_accurate+=model_test(image_label_30[i][0],image_label_30[i][1])
   # process_bar.show_process()
   


In [14]:
print('训练集 正确数:',ADD_accurate)
print('训练集 正确率:',np.round(ADD_accurate/len(image_label_30)*100,2))

训练集 正确数: 6142
训练集 正确率: 48.14


3.1.2 ship

In [4]:
model_ship = keras.models.load_model('ship-model-2023-6-14.h5')

In [5]:
def model_test(image_name,label):
    # 加载测试数据并进行预处理
    i_path='./ship/train/images/'+image_name
    test_data = cv2.resize(cv2.imread(i_path,cv2.IMREAD_GRAYSCALE),(32,32))
    test_data=np.expand_dims(test_data, axis=-1)
    # 调整输入数据的形状
    if len(test_data.shape) == 3:
        # 单个图像，形状为(64, 64, 1)
        test_data = np.expand_dims(test_data, axis=0)
    elif len(test_data.shape) == 4:
        # 多个图像，形状为(N, 64, 64, 1)
        pass  # 数据已经符合要求
    else:
        raise ValueError("Invalid input shape")
    # 进行预测
    predictions = model_ship.predict(test_data, verbose=0)
    # 判断结果正误
    if(predictions.argmax()==int(label)):
        return(1)
    else:
        return(0)

In [6]:
image_label_5=list() # 创建空列表，用于存储图像-标签对

f=open('ship_train_image_label_6_14.txt','r')
for i in f.readlines():
    image_label_5.append(i.split('\n')[0]) # 读取图像-标签对
f.close()
for i in range(len(image_label_5)):
    image_label_5[i]=image_label_5[i].split(' ') # 预处理图像-标签对
# 训练集
ADD_accurate=0
process_bar = ShowProcess(len(image_label_5), 'Processing Finished')
for i in range(len(image_label_5)):
   ADD_accurate+=model_test(image_label_5[i][0],image_label_5[i][1])
   process_bar.show_process()
print('训练集 正确数:',ADD_accurate)
print('训练集 正确率:',np.round(ADD_accurate/len(image_label_5)*100,2))

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>]100.00%
Processing Finished
训练集 正确数: 2986
训练集 正确率: 59.68


In [8]:
image_label_5=list() # 创建空列表，用于存储图像-标签对

f=open('ship_test_image_label_6_14.txt','r')
for i in f.readlines():
    image_label_5.append(i.split('\n')[0]) # 读取图像-标签对
f.close()
for i in range(len(image_label_5)):
    image_label_5[i]=image_label_5[i].split(' ') # 预处理图像-标签对
# 训练集
ADD_accurate=0
process_bar = ShowProcess(len(image_label_5), 'Processing Finished')
for i in range(len(image_label_5)):
   ADD_accurate+=model_test(image_label_5[i][0],image_label_5[i][1])
   process_bar.show_process()
print('测试集 正确数:',ADD_accurate)
print('测试集 正确率:',np.round(ADD_accurate/len(image_label_5)*100,2))


[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>]100.00%
Processing Finished
测试集 正确数: 558
测试集 正确率: 44.68


##### 3.2 量化模型

3.2.1 whale

In [13]:
# 加载TFLite模型
interpreter = tf.lite.Interpreter(model_path='./tflite_model/whale-model-2023-6-14-quantization.tflite')
interpreter.allocate_tensors()

# 获取输入和输出张量的索引
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

def model_test(image_name,label):
    # 加载测试数据并进行预处理
    img_path='./whale64x64/'+image_name
    # input_data = cv2.resize(cv2.imread(img_path,cv2.IMREAD_GRAYSCALE),(64,64))
    input_data = cv2.imread(img_path,cv2.IMREAD_GRAYSCALE).astype('float32')
    test_image=input_data.reshape((1,64,64))
    # 设置输入张量的值
    interpreter.set_tensor(input_details[0]['index'], test_image)
    # 运行模型
    interpreter.invoke()
    # 获取输出张量的值
    output_data = interpreter.get_tensor(output_details[0]['index'])
    # 判断结果正误
    if(output_data.argmax()==int(label)):
        return(1)
    else:
        return(0)

In [14]:
whale_image_label_30=list() # 创建空列表，用于存储图像-标签对

f=open('test_image_label_6_13.txt','r')
for i in f.readlines():
    whale_image_label_30.append(i.split('\n')[0]) # 读取图像-标签对
f.close()
for i in range(len(whale_image_label_30)):
    whale_image_label_30[i]=whale_image_label_30[i].split(' ') # 预处理图像-标签对
# 测试集
ADD_accurate=0
process_bar = ShowProcess(len(whale_image_label_30), 'Processing Finished')
for i in range(len(whale_image_label_30)):
   ADD_accurate+=model_test(whale_image_label_30[i][0],whale_image_label_30[i][1])
   process_bar.show_process()


[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>]100.00%
Processing Finished


In [15]:
print('训练集 正确数:',ADD_accurate)
print('训练集 正确率:',np.round(ADD_accurate/len(whale_image_label_30)*100,2))

训练集 正确数: 2651
训练集 正确率: 20.78


3.2.2 ship

In [3]:
# 加载TFLite模型
# interpreter = tf.lite.Interpreter(model_path='./tflite_model/ship-model-2023-6-14-quantization.tflite')
interpreter = tf.lite.Interpreter(model_path='D:/workshop/2023-6-12/5-Version/2-ship/model/ship-model-2023-6-14-quantization.tflite')
interpreter.allocate_tensors()

# 获取输入和输出张量的索引
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()



In [4]:
def model_test(image_name,label):
    # 加载测试数据并进行预处理
    # img_path='./ship/train/images/'+image_name
    img_path='./ship-resize32/'+image_name
    input_data = cv2.resize(cv2.imread(img_path,cv2.IMREAD_GRAYSCALE),(32,32)).astype('float32')
    test_image=input_data.reshape((1,32,32))
    # 设置输入张量的值
    interpreter.set_tensor(input_details[0]['index'], test_image)
    # 运行模型
    interpreter.invoke()
    # 获取输出张量的值
    output_data = interpreter.get_tensor(output_details[0]['index'])
    # 判断结果正误
    if(output_data.argmax()==int(label)):
        return(1)
    else:
        return(0)

In [6]:
ship_image_label_5=list() # 创建空列表，用于存储图像-标签对

f=open('ship_train_image_label_6_14.txt','r')
for i in f.readlines():
    ship_image_label_5.append(i.split('\n')[0]) # 读取图像-标签对
f.close()
for i in range(len(ship_image_label_5)):
    ship_image_label_5[i]=ship_image_label_5[i].split(' ') # 预处理图像-标签对

ADD_accurate=0
process_bar = ShowProcess(len(ship_image_label_5), 'Processing Finished')
for i in range(len(ship_image_label_5)):
   ADD_accurate+=model_test(ship_image_label_5[i][0],ship_image_label_5[i][1])
   process_bar.show_process()
print('训练集 正确数:',ADD_accurate)
print('训练集 正确率:',np.round(ADD_accurate/len(ship_image_label_5)*100,2))


[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>]100.00%
Processing Finished
训练集 正确数: 3000
训练集 正确率: 59.96


In [7]:
ship_image_label_5=list() # 创建空列表，用于存储图像-标签对

f=open('ship_test_image_label_6_14.txt','r')
for i in f.readlines():
    ship_image_label_5.append(i.split('\n')[0]) # 读取图像-标签对
f.close()
for i in range(len(ship_image_label_5)):
    ship_image_label_5[i]=ship_image_label_5[i].split(' ') # 预处理图像-标签对

ADD_accurate=0
process_bar = ShowProcess(len(ship_image_label_5), 'Processing Finished')
for i in range(len(ship_image_label_5)):
   ADD_accurate+=model_test(ship_image_label_5[i][0],ship_image_label_5[i][1])
   process_bar.show_process()
print('测试集 正确数:',ADD_accurate)
print('测试集 正确率:',np.round(ADD_accurate/len(ship_image_label_5)*100,2))


[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>]100.00%
Processing Finished
测试集 正确数: 551
测试集 正确率: 44.12


### 分割线=================

In [ ]:
# 加载测试数据并进行预处理
i_path='./cropped_train_images/'+image_name
test_data = cv2.resize(cv2.imread(i_path,cv2.IMREAD_GRAYSCALE),(64,64))
test_data=np.expand_dims(test_data, axis=-1)
# 调整输入数据的形状
if len(test_data.shape) == 3:
    # 单个图像，形状为(64, 64, 1)
    test_data = np.expand_dims(test_data, axis=0)
elif len(test_data.shape) == 4:
    # 多个图像，形状为(N, 64, 64, 1)
    pass  # 数据已经符合要求
else:
    raise ValueError("Invalid input shape")
# 进行预测
predictions = model.predict(test_data, verbose=0)

### 3.2 量化后测试

In [ ]:
# 加载量化后的TFLite模型
interpreter = tf.lite.Interpreter(model_path='whale-model-2023-6-10-quantization.tflite')
interpreter.allocate_tensors()

# 获取输入和输出张量的索引
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# 读取测试数据-10张手写数字图片-预处理
num_image=list()
for i in range(10):
    img_path='handwrite_num/{}.png'.format(i)
    num_image.append(cv2.resize(cv2.imread(img_path,cv2.IMREAD_GRAYSCALE),(64,64))/255)
input_data = np.array(num_image)

for i in range(10):
    test_image=input_data[i].reshape((1,227,227,3))
    # 设置输入张量的值
    interpreter.set_tensor(input_details[0]['index'], test_image)
    # 运行模型
    interpreter.invoke()
    # 获取输出张量的值
    output_data = interpreter.get_tensor(output_details[0]['index'])
    print('num',i,'预测结果',np.argmax(output_data))

### ================

In [4]:
image_label_30=list() # 创建空列表，用于存储图像-标签对

f=open('test_image_label_6_13.txt','r')
for i in f.readlines():
    image_label_30.append(i.split('\n')[0]) # 读取图像-标签对
f.close()
for i in range(len(image_label_30)):
    image_label_30[i]=image_label_30[i].split(' ') # 预处理图像-标签对

In [113]:
def model_test(image_name,label):
    # 加载测试数据并进行预处理
    i_path='./cropped_train_images/'+image_name
    test_data = cv2.resize(cv2.imread(i_path,cv2.IMREAD_GRAYSCALE),(64,64))
    test_data=np.expand_dims(test_data, axis=-1)
    # 调整输入数据的形状
    if len(test_data.shape) == 3:
        # 单个图像，形状为(64, 64, 1)
        test_data = np.expand_dims(test_data, axis=0)
    elif len(test_data.shape) == 4:
        # 多个图像，形状为(N, 64, 64, 1)
        pass  # 数据已经符合要求
    else:
        raise ValueError("Invalid input shape")
    # 进行预测
    predictions = model.predict(test_data, verbose=0)
    # 判断结果正误
    if(predictions.argmax()==int(label)):
        return(1)
    else:
        return(0)

In [7]:
# 测试集
ADD_accurate=0
process_bar = ShowProcess(len(image_label_30), 'Processing Finished')
for i in range(len(image_label_30)):
   ADD_accurate+=model_test(image_label_30[i][0],image_label_30[i][1])
   process_bar.show_process()
   


[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>]100.00%
Processing Finished


In [11]:
print('测试集 准确数',ADD_accurate)
print('测试集 准确率',np.round(ADD_accurate/len(image_label_30),2))

测试集准确数 6152
测试集准确率 0.48


In [5]:
image_label_30=list() # 创建空列表，用于存储图像-标签对

f=open('train_image_label_6_13.txt','r')
for i in f.readlines():
    image_label_30.append(i.split('\n')[0]) # 读取图像-标签对
f.close()
for i in range(len(image_label_30)):
    image_label_30[i]=image_label_30[i].split(' ') # 预处理图像-标签对

In [12]:
# 训练集 前半
ADD_accurate=0
process_bar = ShowProcess(int(len(image_label_30)/2), 'Processing Finished')
for i in range(int(len(image_label_30)/2)):
   ADD_accurate+=model_test(image_label_30[i][0],image_label_30[i][1])
   process_bar.show_process()

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>]100.00%
Processing Finished


In [13]:
print('训练集-前半 准确数',ADD_accurate)
print('训练集-前半 准确率',np.round(ADD_accurate/int(len(image_label_30)/2)*100,2))

训练集-前半 准确数 10733
训练集-前半 准确率 0.56


In [6]:
# 训练集 后半
ADD_accurate=0
process_bar = ShowProcess(int(len(image_label_30)/2), 'Processing Finished')
for i in range(int(len(image_label_30)/2)):
   ADD_accurate+=model_test(image_label_30[i+int(len(image_label_30)/2)][0],image_label_30[i+int(len(image_label_30)/2)][1])
   process_bar.show_process()

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>]100.00%
Processing Finished


In [7]:
print('训练集-后半 准确数',ADD_accurate)
print('训练集-后半 准确率',np.round(ADD_accurate/int(len(image_label_30)/2)*100,2))

训练集-后半 准确数 10708
训练集-后半 准确率 0.56


In [13]:
print('训练集 准确数',10733+10708)
print('训练集 准确率',np.round((10733+10708)/len(image_label_30)*100,2),'%')

训练集 准确数 21441
训练集 准确率 56.02 %


### ==========分割线=============

In [5]:
name=['melon_headed_whale' # 已经分出的29个物种
,'humpback_whale'
,'false_killer_whale'
,'bottlenose_dolphin'
,'beluga'
,'minke_whale'
,'fin_whale'
,'blue_whale'
,'gray_whale'
,'southern_right_whale'
,'common_dolphin'
,'kiler_whale'
,'pilot_whale'
,'dusky_dolphin'
,'killer_whale'
,'long_finned_pilot_whale'
,'sei_whale'
,'spinner_dolphin'
,'cuviers_beaked_whale'
,'spotted_dolphin'
,'globis'
,'brydes_whale'
,'commersons_dolphin'
,'white_sided_dolphin'
,'short_finned_pilot_whale'
,'rough_toothed_dolphin'
,'pantropic_spotted_dolphin'
,'pygmy_killer_whale'
,'frasiers_dolphin']


In [57]:
final_list=list()
for i in range(29):
    f=open('./species-test/{}.txt'.format(name[i]),'r')
    for speceis_image in f.readlines():
        speceis_image=speceis_image.split('\n')[0]
        if model_test(speceis_image.split('\n')[0],str(i))==1:
            print(i,speceis_image,name[i])
            # final_list.append(speceis_image)
            break
        else:
            continue
        # else:
        #     print('not found in species[{}]'.format(name[i]))
    f.close()
    

0 93d7d0699a1216.jpg melon_headed_whale
1 54b9dba2604930.jpg humpback_whale
2 2729fe7ad6245f.jpg false_killer_whale
3 6c39c44f811efe.jpg bottlenose_dolphin
4 d4135027a5647b.jpg beluga
5 67338191a6d07d.jpg minke_whale
6 d497600106cae2.jpg fin_whale
7 d7f47c7a713f91.jpg blue_whale
8 d831c5d3af3342.jpg gray_whale
9 eafd6677c18db7.jpg southern_right_whale
11 b69fb176f39fa0.jpg kiler_whale
12 1707e63f92f03f.jpg pilot_whale
13 cdfa1a63ed125a.jpg dusky_dolphin
14 a0b9d49c67f664.jpg killer_whale
15 6f6108b449daab.jpg long_finned_pilot_whale
16 12df50e19a90ab.jpg sei_whale
17 9b6e59ff6d2309.jpg spinner_dolphin
18 35770005852302.jpg cuviers_beaked_whale
19 975914ab797f11.jpg spotted_dolphin
20 9443c5069ac1df.jpg globis
22 c2a9fdd5401ebb.jpg commersons_dolphin
23 19d93b543ff684.jpg white_sided_dolphin
24 76592eacf0b914.jpg short_finned_pilot_whale
27 b40969974439be.jpg pygmy_killer_whale


In [125]:
final_list_plus=[10,21,25,26,28]

for i in range(5):
    f=open('./species-train/{}.txt'.format(name[final_list_plus[i]]),'r')
    for speceis_image in f.readlines():
        speceis_image=speceis_image.split('\n')[0]
        if model_test(speceis_image.split('\n')[0],final_list_plus[i]):
            print(final_list_plus[i],speceis_image,name[i])
            # final_list_plus[i]=speceis_image
            break
    f.close()


10 0938bb32569bed.jpg melon_headed_whale
21 7a7d0e488c79c6.jpg humpback_whale
25 11bbf6dc02655d.jpg false_killer_whale
26 703317a1fca5c7.jpg bottlenose_dolphin
28 c37aef077d41d1.jpg beluga
